

QUESTIONS:
What is our 30-day all cause readmission rate across hospitals our members are admitted to?
What percent of patients have a PCP visit within a week of discharge?
Do either vary by individual or group PCP's?

BONUS:
Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume?
What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?
Are there certain patients we should focus on (e.g. frequent fliers)?
From this, what could a version 1 of an actionable dashboard or intervention list look like to support the Transition Care Managers?

NOTES:

discharge date + 30 days: readmission true or false
count trues

In [2]:
import pandas as pd
import datetime as dt

In [62]:
members = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv', parse_dates=[61,62])
admissions = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12])

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [63]:
admissions_dedup = admissions.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
admissions_dedup = admissions_dedup.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
admissions.shape, admissions_dedup.shape

((6213, 23), (6149, 23))

In [64]:
def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    readmission = False
    for ind, row in patient_admissions.iterrows():
        ##set first event as an intiial event or a non-qualifying readmission as initial event
        if patient_index_events == 0 or row['Encounter Start Date'] > last_index_event_discharge_date + dt.timedelta(days=30):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter End Date']
            readmission = False
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'index'
        ##check if qualifying readmission and change readmit status of most recently added initial event to true
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'readmission'
        ##check for duplicate readmission
        else:
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'duplicate readmission'
            
    return patient_index_events, patient_readmission_events
    

In [65]:
total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in admissions_dedup['Patient ID'].unique():
    patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        encounter_id = patient_admissions['Encounter ID'].values[0]
        admissions.loc[admissions['Encounter ID'] == encounter_id, 'type'] = 'index'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

<ipython-input-65-b229257dddde>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
<ipython-input-65-b229257dddde>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)


Total Index Events: 5536
Index Events Leading to Readmission: 527
Readmission Rate: 9.52%


In [66]:
admissions['type'].value_counts()

index                    5536
readmission               527
duplicate readmission      86
Name: type, dtype: int64